## Installing Dependencies

In [1]:
!pip install gTTS
!pip install pydub
!pip install pandas
!pip install moviepy
!pip install Pillow
!pip install imageio==2.4.1
!pip install --upgrade imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 4.4 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 7.3 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=24e51b4acf9c05127cf31c602db2f5e4e467dbb42ae166aace78a80e6846c824
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Look

In [2]:
# Creating temporary folders
!mkdir img
!mkdir temp
!mkdir mp3
!mkdir font
!mkdir video

In [3]:
# Importing Libraries
from gtts import gTTS
from pydub import AudioSegment
import pandas as pd
from moviepy.editor import AudioFileClip, ImageClip, VideoClip
from PIL import Image, ImageDraw, ImageFont

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2916352/45929032 bytes (6.3%)6643712/45929032 bytes (14.5%)10272768/45929032 bytes (22.4%)13910016/45929032 bytes (30.3%)17498112/45929032 bytes (38.1%)20979712/45929032 bytes (45.7%)24436736/45929032 bytes (53.2%)27992064/45929032 bytes (60.9%)31686656/45929032 bytes (69.0%)35454976/45929032 bytes (77.2%)39149568/45929032 bytes (85.2%)42876928/45929032 bytes (93.4%)

## Extracting Phrases

In [4]:
# Loading csv into pandas dataframe
df = pd.read_csv('/content/phrases.csv')

# Creating Korean and English text list from datafram
input_text = df['Korean'].tolist()
input_translation = df['English'].tolist()

# Setting Language for TTS
language = 'ko' # zh = Chinese, en = English, ko = Korean

## Generating TTS Audio Files and Combining

Audio Files in Korean normal speed, Korean slow speed, and English. Combines 3 files into single mp3 file

In [5]:
# Generating Korean and English TTS Audio and combining them into an mp3 file
def generateAudio(): 
    for i in range(len(input_text)):

        file_name = input_text[i]

        normal = gTTS(text = input_text[i], lang=language, slow=False)
        audio_normal = f'/content/temp/{file_name}_normal.mp3'
        normal.save(audio_normal)

        slow = gTTS(text = input_text[i], lang=language, slow=True)
        audio_slow = f'/content/temp/{file_name}_slow.mp3'
        slow.save(audio_slow)

        english = gTTS(text = input_translation[i], lang='en', slow=False)
        audio_english = f'/content/temp/{file_name}_english.mp3'
        english.save(audio_english)

        playlist_songs = [AudioSegment.from_mp3(audio_normal), AudioSegment.silent(duration=1000), AudioSegment.from_mp3(audio_slow), AudioSegment.silent(duration=1000), AudioSegment.from_mp3(audio_english),AudioSegment.silent(duration=1000)]
        combined = AudioSegment.empty()
        for song in playlist_songs:
            combined += song
        combined.export(f'/content/mp3/{file_name}.mp3', format="mp3")
        
generateAudio()

## Generating Images

Generating images with Korean and English text

In [9]:
def genImage():
    width = 1080
    height = 1080
    font = ImageFont.truetype("/content/font/NotoSerifKR-Regular.otf", size=50)
    i = 0
    for line in input_text:
        file_name = input_text[i]
        message = input_text[i] + "\n" + input_translation[i]
        i += 1
        img = Image.new('RGB', (width, height), color='#FFDDD2')
        imgDraw = ImageDraw.Draw(img)

        textWidth, textHeight = imgDraw.textsize(message, font=font)
        xText = (width - textWidth) / 2
        yText = (height - textHeight) / 2

        imgDraw.text((xText, yText), message, font=font, fill=(89, 69, 69))
        img.save(f'/content/img/{file_name}.png')
genImage()

## Combines images and audio to generate video

In [10]:
def mp3PNGMerge():

  for i in range(len(input_text)):
    file_name = input_text[i]
    audio_clip = AudioFileClip(f"/content/mp3/{file_name}.mp3")
    image_clip = ImageClip(f"/content/img/{file_name}.png")
    video_clip = image_clip.set_audio(audio_clip)
    video_clip.duration = audio_clip.duration
    video_clip.fps = 30
    video_clip.write_videofile(f'/content/video/{file_name}.mp4')

mp3PNGMerge()

[MoviePy] >>>> Building video /content/video/영어하세요?.mp4
[MoviePy] Writing audio in 영어하세요?TEMP_MPY_wvf_snd.mp3


100%|██████████| 165/165 [00:00<00:00, 1147.92it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/video/영어하세요?.mp4



100%|██████████| 224/224 [00:03<00:00, 61.65it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/영어하세요?.mp4 

[MoviePy] >>>> Building video /content/video/깎아주세요..mp4
[MoviePy] Writing audio in 깎아주세요.TEMP_MPY_wvf_snd.mp3


100%|██████████| 168/168 [00:00<00:00, 911.23it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/video/깎아주세요..mp4



100%|██████████| 228/228 [00:03<00:00, 70.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/깎아주세요..mp4 

